In [66]:
import pandas as pd
import MeCab, re
tagger = MeCab.Tagger()

# replace \n+ with ''
# 0:surface form, 1:pos, 2:pos2(サ変接続, etc), 3:pos3, 4:pos4
# 5:conjugation type, 6:conjugation form, 7:lemma, 8:phono, 9:phone

def tokenize_mecab(text:str) -> list:
    text = re.sub(r'\n+', '\n', text)
    tokens = tagger.parse(text).split('\n')[:-2]
    return [re.split(r'[,\t]', token) for token in tokens]

def get_verb_group(conjtype:str) -> str:
    """
    >>> get_verb_group('五段・カ行')
    '1'
    """
    return {'五段':'1', '一段':'2', 'サ変':'3', 'カ変':'3'}.get(conjtype[:2], '')

def tokenize(text:str) -> list:
    text = re.sub(r'\n+', '\n', text)
    tokens = tagger.parse(text).split('\n')[:-2]
    tokens = [re.split(r'[,\t]', token) for token in tokens]
    surface, lemma, pos = ['<SEP>'],['<SEP>'],['<SEP>']
    
    i = 0
    while i < len(tokens):
        if tokens[i][2] == 'サ変接続' and tokens[i+1][2] == 'サ変接続' and tokens[i+2][7] == 'する':
            surface.append(tokens[i][0] + tokens[i+1][0] + tokens[i+2][0]) # surface form
            lemma.append(tokens[i][7] + tokens[i+1][7] + tokens[i+2][0]) # lemma
            pos.append('動詞3') # pos
            i += 2
        elif tokens[i][2] == 'サ変接続' and tokens[i+1][7] == 'する':
            surface.append(tokens[i][0] + tokens[i+1][0]) # surface form
            lemma.append(tokens[i][7] + tokens[i+1][7]) # lemma
            pos.append('動詞3') # pos
            i += 2
        else:
            surface.append(tokens[i][0]) # surface form
            lemma.append(tokens[i][7]) # lemma
            pos.append(tokens[i][1]) # pos  
            if tokens[i][0] == '。':
                surface.append('<SEP>')
                lemma.append('<SEP>')
                pos.append('<SEP>')
            i += 1
    return surface, lemma, pos

In [67]:
tokenize_mecab('運動させる際は予防対策を選挙活動するよう求めています。')

[['運動', '名詞', 'サ変接続', '*', '*', '*', '*', '運動', 'ウンドウ', 'ウンドー'],
 ['さ', '動詞', '自立', '*', '*', 'サ変・スル', '未然レル接続', 'する', 'サ', 'サ'],
 ['せる', '動詞', '接尾', '*', '*', '一段', '基本形', 'せる', 'セル', 'セル'],
 ['際', '名詞', '非自立', '副詞可能', '*', '*', '*', '際', 'サイ', 'サイ'],
 ['は', '助詞', '係助詞', '*', '*', '*', '*', 'は', 'ハ', 'ワ'],
 ['予防', '名詞', 'サ変接続', '*', '*', '*', '*', '予防', 'ヨボウ', 'ヨボー'],
 ['対策', '名詞', 'サ変接続', '*', '*', '*', '*', '対策', 'タイサク', 'タイサク'],
 ['を', '助詞', '格助詞', '一般', '*', '*', '*', 'を', 'ヲ', 'ヲ'],
 ['選挙', '名詞', 'サ変接続', '*', '*', '*', '*', '選挙', 'センキョ', 'センキョ'],
 ['活動', '名詞', 'サ変接続', '*', '*', '*', '*', '活動', 'カツドウ', 'カツドー'],
 ['する', '動詞', '自立', '*', '*', 'サ変・スル', '基本形', 'する', 'スル', 'スル'],
 ['よう', '名詞', '非自立', '助動詞語幹', '*', '*', '*', 'よう', 'ヨウ', 'ヨー'],
 ['求め', '動詞', '自立', '*', '*', '一段', '連用形', '求める', 'モトメ', 'モトメ'],
 ['て', '助詞', '接続助詞', '*', '*', '*', '*', 'て', 'テ', 'テ'],
 ['い', '動詞', '非自立', '*', '*', '一段', '連用形', 'いる', 'イ', 'イ'],
 ['ます', '助動詞', '*', '*', '*', '特殊・マス', '基本形', 'ます', 'マス', 'マス'],


In [68]:
tokenize('運動させる際は予防対策を選挙活動するよう求めています。')

(['<SEP>',
  '運動さ',
  'せる',
  '際',
  'は',
  '予防',
  '対策',
  'を',
  '選挙活動する',
  'する',
  'よう',
  '求め',
  'て',
  'い',
  'ます',
  '。',
  '<SEP>'],
 ['<SEP>',
  '運動する',
  'せる',
  '際',
  'は',
  '予防',
  '対策',
  'を',
  '選挙活動する',
  'する',
  'よう',
  '求める',
  'て',
  'いる',
  'ます',
  '。',
  '<SEP>'],
 ['<SEP>',
  '動詞3',
  '動詞',
  '名詞',
  '助詞',
  '名詞',
  '名詞',
  '助詞',
  '動詞3',
  '動詞',
  '名詞',
  '動詞',
  '助詞',
  '動詞',
  '助動詞',
  '記号',
  '<SEP>'])

In [40]:
data = pd.read_json('nhkweb2013.json')
data = data[['id', 'title', 'article', 'genre', 'datePublished']]
data['article'] = data.article.apply(lambda x:re.sub(r'\n+', '', x))
#data['datePublished'] = data.datePublished.apply(lambda x:x.split('T')[0])
data

,id,title,article,genre,datePublished
0,k10013005311000,運動時の熱中症予防に指針,今週、学校の授業や行事の最中に、子どもたちが熱中症の症状を訴えるケースが相次いでいます。文部...,[社会],2013-07-12T17:25
1,k10013902851000,新型ロケット イプシロンを初公開,１２年ぶりに開発された国産の新型ロケットイプシロンが、今月２７日に打ち上げられるのを前に、鹿...,[科学・文化],2013-08-20T17:17
2,k10013919831000,三國連太郎さん死去 「飢餓海峡」「釣りバカ日誌」など,「飢餓海峡」や「釣りバカ日誌」シリーズなど数々の映画で活躍した日本を代表する演技派俳優の三國...,[科学・文化],2013-04-15T14:18
3,k10013009451000,ＥＶ充電設備の設置支援始まる,国内の主な自動車メーカー４社は電気自動車の普及に向け、高額な負担が足かせとなっている充電設備...,[ビジネス],2013-11-13T07:24
4,k10013011981000,政府･与党 違憲判決で区割りの改定急ぐ,東京高等裁判所が、去年１２月の衆議院選挙の１票の格差を憲法違反だとする判決を言い渡したことを...,[政治],2013-03-07T04:32
...,...,...,...,...,...
192,k10015855551000,「ジェットマン」富士山を飛ぶ,小型ジェットエンジンが付いた翼を身につけて世界各地の空を飛んでいるスイス人の冒険家が、富士山...,[科学・文化],2013-11-06T21:59
193,k10015877601000,アシアナ機 「失速して着陸やり直し」,アメリカ・サンフランシスコの空港で、韓国のアシアナ航空の旅客機が着陸に失敗して炎上し、２人が...,[国際],2013-07-08T07:47
194,k10015878911000,技能五輪 日本勢はメダル１２個,世界各地の若い技術者たちが仕事の技能を競い合う「技能五輪国際大会」がドイツで開かれ、日本は出...,[国際],2013-07-08T09:41
195,k10015879641000,重い生理痛 子宮内膜症のおそれも,重い生理痛を放っておくと子宮内膜症などの病気に進行するおそれがあるとして、医師などで作るグル...,[科学・文化],2013-11-07T17:35


In [41]:
data.article[0]

'今週、学校の授業や行事の最中に、子どもたちが熱中症の症状を訴えるケースが相次いでいます。文部科学省は、学校での熱中症のほとんどが体育の授業などスポーツ活動の際に起きていることから、全国の小中学校などに指針を示し、運動させる際は予防対策を徹底するよう求めています。それによりますと、気温が３０度以下であっても湿度が高いときは熱中症のリスクが増すとして、運動する際は、室内外を問わず、必要に応じて水分や塩分を補給できる環境を整えるようにするとしています。また、梅雨明けなど急に暑くなったときは運動を軽くし、徐々に暑さに慣らしていくことや、運動をする際は３０分程度に１回は休憩を取ること、さらには、万が一に備えて、教職員に対し応急手当ての研修を実施することなどとしています。文部科学省はこうした指針を基に子どもの体力や体調などを考慮し、各学校で臨機応変に判断するよう求めていて、教職員が適切に対応できるよう熱中症に関する知識を身に付けてほしいとしています。文部科学省によりますと、平成２３年までのおよそ２０年間に、全国の小中学校や高校で熱中症によって死亡した児童や生徒は７５人で、このうち９割にあたる６９人が体育活動中に熱中症になっているということです。連日、猛烈な暑さが続くなか、東京・北区の中学校は子どもたちをできる限り炎天下で活動させないよう学校運営に気を配っています。全校生徒４００人余りの区立稲付中学校では、生徒に水筒を持参させ、給食の時間などに校内放送を行い、こまめな水分補給を呼びかけています。廊下には温度計を設置して、校長や教員が定期的にチェックし、３１度以上になると激しい運動を中止、３５度以上では運動自体を原則、中止しています。１２日は校長が部活動の顧問の教員を集め熱中症に注意するよう呼びかけ、必ず、１５分から３０分おきに生徒に休憩をとらせることや、熱中症が疑われる場合は活動を中止することなどを確認しました。この学校では、熱中症を防ぐため、気温が高くなるこの時期に運動会などの行事を組まず、暑くなる前の時期や秋になってから行うことにしています。さらに屋外で体育の授業を行う際も気温が３５度を超えた時点で室内での活動に切り替えるなど、子どもたちをできる限り炎天下で活動させないよう、学校運営に気を配っています。しかし、ことしは例年より早く猛暑となり、生徒が暑さに慣れていないうえ、気

In [51]:
newdf = pd.DataFrame(columns=list(data.columns) + ['unlemma', 'lemma', 'pos'])
for i, row in data.iterrows():
    tokens = tokenize(row['article'])
    newdf.loc[i] = list(row) + list(tokens)

In [52]:
newdf[['unlemma']].iat[0,0]

['<SEP>',
 '今週',
 '、',
 '学校',
 'の',
 '授業',
 'や',
 '行事',
 'の',
 '最中',
 'に',
 '、',
 '子ども',
 'たち',
 'が',
 '熱中',
 '症',
 'の',
 '症状',
 'を',
 '訴える',
 'ケース',
 'が',
 '相次い',
 'で',
 'い',
 'ます',
 '。',
 '<SEP>',
 '文部',
 '科学',
 '省',
 'は',
 '、',
 '学校',
 'で',
 'の',
 '熱中',
 '症',
 'の',
 'ほとんど',
 'が',
 '体育',
 'の',
 '授業',
 'など',
 'スポーツ',
 '活動',
 'の',
 '際',
 'に',
 '起き',
 'て',
 'いる',
 'こと',
 'から',
 '、',
 '全国',
 'の',
 '小中学校',
 'など',
 'に',
 '指針',
 'を',
 '示し',
 '、',
 '運動',
 'さ',
 'せる',
 '際',
 'は',
 '予防',
 '対策',
 'を',
 '徹底',
 'する',
 'よう',
 '求め',
 'て',
 'い',
 'ます',
 '。',
 '<SEP>',
 'それ',
 'に',
 'より',
 'ます',
 'と',
 '、',
 '気温',
 'が',
 '３',
 '０',
 '度',
 '以下',
 'で',
 'あっ',
 'て',
 'も',
 '湿度',
 'が',
 '高い',
 'とき',
 'は',
 '熱中',
 '症',
 'の',
 'リスク',
 'が',
 '増す',
 'として',
 '、',
 '運動',
 'する',
 '際',
 'は',
 '、',
 '室内',
 '外',
 'を',
 '問わ',
 'ず',
 '、',
 '必要',
 'に',
 '応じ',
 'て',
 '水分',
 'や',
 '塩分',
 'を',
 '補給',
 'できる',
 '環境',
 'を',
 '整える',
 'よう',
 'に',
 'する',
 'と',
 'し',
 'て',
 'い',
 'ます',
 '。',
 '<SEP>',
 'また',
 '、',
 '梅雨',
 '明け'

In [45]:
i = 5
print(len(newdf.loc[i].unlemma))
print(len(newdf.loc[i].lemma))
print(len(newdf.loc[i].pos))

335
335
335
